In [1]:
from supabase.client import Client
from typing import Dict, Any, Tuple, List
from langchain import hub
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores.supabase import SupabaseVectorStore
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain.tools.retriever import create_retriever_tool
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai import ChatOpenAI
from langchain.utilities.tavily_search import TavilySearchAPIWrapper
from langchain.tools.tavily_search import TavilySearchResults
from langchain.tools import DuckDuckGoSearchResults
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
import json

from utils.utils import initialize_environment_variables, initialize_subabase_client, initialize_openai_client
from utils.utils import initialize_tavily_client
from api.config import interface_config

def unpack_prompt_input(input: Dict[str, Any]) -> Tuple[str, str, str, str, List[str], List[str]]:
    school_type = str(input.get("subject"))
    subject = str(input.get("subject"))
    topic = str(input.get("topic"))
    grade = str(input.get("grade"))
    state = str(input.get("state"))
    keywords = [str(i) for i in input.get("keywords")]
    context = [str(i) for i in input.get("context")]
    return school_type, subject, topic, grade, state, keywords, context

def create_output_table_format():
    return """{{
    "learn_goals": "xyz",
    "table_data": [
        {{"title": "abc", "duration": "10min", "content": "blabla"}},
        {{...}},
        {{...}}
    ]
}}"""

json_output = create_output_table_format()

def create_task_string(school_type: str, subject: str, topic: str, grade: str, state: str, json_output: str) -> str:
    return f"""
Schritt 1: Plane eine Unterrichtsstunde im Fach {subject} zum Thema {topic} für eine {grade} Klasse in {state} für den Schultyp {school_type}!
Schritt 2: Formatiere deine Antwort in der folgenden json-Struktur:
______________________________
{json_output}
______________________________

"""


def get_uploaded_ids(supabase_client, filepath, upload_table_name):
    data, _cnt = supabase_client.table(upload_table_name).select('id').contains('metadata', {'source':filepath}).execute()
    return [data[1][i].get("id") for i in range(len(data[1]))]


def create_filter_list(context):
    uploaded_ids = []
    for doc in context:
        if len(doc) > 0:
            doc_uploaded_ids = get_uploaded_ids(doc, interface_config.upload_table_name)
            if len(doc_uploaded_ids) > 0:
                uploaded_ids += doc_uploaded_ids
    return uploaded_ids


def initialize_vector_store(client: Client, embedding: OpenAIEmbeddings, table_name: str, query_name: str) -> SupabaseVectorStore:
    vector_store = SupabaseVectorStore(
        client=client,
        embedding=embedding,
        table_name=table_name,
        query_name=query_name,
    )
    return vector_store


def create_prompt_template(retrieved_context: List[Any], keywords, topic) -> str:
    keywords = '\n   -'.join(keywords)
    return f"""
Du bist ein Assistent für Lehrkräfte und deine Aufgabe ist Unterricht strukturiert, detailliert und fachlich korrekt vorzubereiten.

Lernziele:
------------------
- Thema: {topic}
- Schwerpunkte:
    {keywords}
------------------

Die Lehrkraft stellt dir Kontext zur Lösung der Aufgabe in deiner Bibliothek zur Verfügung. 
Hier ist ein kurzer Ausschnitt aus dem Kontext in deiner Bibliothek:

Ausschnitt aus dem Kontext:
---------
{retrieved_context}
---------

Die Gesamtlänge des Unterrichts soll insgesamt 45 Minuten betragen.
Alle Schritte der Aufgabe müssen ausgeführt werden.
Deine Antwort soll ausschließlich aus der beüllten JSON-Tabelle bestehen.

Deine Aufgabe lautet:
"""

def get_answer(prompt_input):
    initialize_environment_variables("../.env")

    # Initialize Supabase Client
    supabase_client = initialize_subabase_client()

    # Initialize OpenAI Client
    _openai_client = initialize_openai_client()

    # Initialize Tavily Client
    _tavily_client = initialize_tavily_client()    
    
    school_type, subject, topic, grade, state, keywords, context = unpack_prompt_input(prompt_input)

    task = create_task_string(school_type, subject, topic, grade, state)

    embedding = OpenAIEmbeddings()
    vector_store = initialize_vector_store(supabase_client, embedding, interface_config.upload_table_name, interface_config.supabase_match_function)

    retrieved_context = vector_store.similarity_search(task, k=interface_config.context_k, filter=context)

    prompt_template = create_prompt_template(retrieved_context, keywords, topic, task)

    prompt = hub.pull("hwchase17/openai-functions-agent")

    search = TavilySearchAPIWrapper()
    tavily_tool = TavilySearchResults(api_wrapper=search, max_results=interface_config.tavily_max_response)

    wrapper = DuckDuckGoSearchAPIWrapper(region="de-de", time="d", max_results=5)
    search = DuckDuckGoSearchResults(api_wrapper=wrapper)

    retriever = vector_store.as_retriever(search_type=interface_config.retriever_search_type, 
                                          search_kwargs={
                                              "k": interface_config.retriever_k,
                                              'lambda_mult': interface_config.retriever_mult, 
                                              'filter': {'uuid': context}
                                                }
                                            )

    retriever_tool = create_retriever_tool(
    retriever,
    "get_more_context",
    "A personal library optimized for user defined context information. Useful for when you need to know more about the given context. Input should be a search query.",
    )

    tools = [retriever_tool, tavily_tool]

    llm = ChatOpenAI(model_name=interface_config.gpt_model_version, temperature=interface_config.gpt_model_temperature)
    agent = create_openai_functions_agent(llm, tools, prompt)

    agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

    response = agent_executor.invoke({"input": prompt_template})

    output = {}
    output["output"] = response["output"]

    output = json.dumps(output)

    return output



In [2]:
input_dict_1 = {
"school_type": "Gymnasium",
"subject": "English",
"topic": "Shakespeare",
"grade": "13.",
"state": "Bayern",
"keywords": ["shakespeare", "poetry", "analysis"], 
"context": ["2147550a-800f-4fd1-9021-e5f443c97ce5", "98e1f41e-5985-418b-a067-aad97091ac23", "26392f41-80ab-4c01-a408-4faf36f06291"]
}

In [3]:
input_dict_2 = {
  "state": "Bayern",
  "school_type": "Realschule",
  "grade": "7",
  "topic": "Napoleon",
  "subject": "Geschichte",
  "keywords": [
    "französische Revolution"
  ],
  "context": [
    "c5ee1bea-3dc3-44bd-8c10-369fe47744dc",
    "6d97b369-25e4-478b-bc6a-3b931b1133fc",
    "06a4701f-fff3-4f50-9d99-0f83945fca0e",
    "b509bf63-d79f-435a-8315-9dd4164cb1e0",
    "11412fb3-eac4-4282-afac-b2de102df6e2",
    "9b51b0b5-d49a-489f-b5a9-04ac6f26185c",
    "7597c3cf-6d4e-4d8a-8a42-759f563c7557",
    "47b7dfd0-c66a-4fca-82af-091a90bfa40e",
    "9e642aa5-6b41-4b49-b626-58fec025094a",
    "5365cae4-25b4-4909-9a24-2f698e51a4cc",
    "b5133c6d-9586-4b4f-a2ed-247babbd1ba8",
    "2694f0ea-f451-4330-bdaf-18f1d5cb20de",
    "5814231e-7ca8-434a-95d7-909ed7022ec0",
    "2147550a-800f-4fd1-9021-e5f443c97ce5",
    "98e1f41e-5985-418b-a067-aad97091ac23",
    "26392f41-80ab-4c01-a408-4faf36f06291",
    "5d026545-f195-4cb9-9394-784542dd9650",
    "12e9cd2c-9c48-4724-9ba5-aaf41436852d",
    "3aecdb10-7e57-46ee-80dc-2a2400d8be4d",
    "4192f1d0-171a-45b1-b4da-db5e12953d70",
    "10e3d8ca-aae4-409f-85cb-aa679c6cd004",
    "1aef0ce7-b6aa-4585-9c75-d890d67832b1",
    "238b2e27-22d6-4911-9962-02256139c1c4",
    "79dba1a5-343a-4e75-8272-72cdd416b8ca",
    "ba80b4e9-c615-4517-88e9-65adb6ad1323"
  ]
}

In [4]:
test = get_answer(input_dict_2)

2024-02-03 19:21:37,716:INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-02-03 19:21:37,950:INFO - HTTP Request: POST https://smxwnqdpxcvclxhbvhsf.supabase.co/rest/v1/rpc/match_documents_with_uuid_filter?limit=1 "HTTP/1.1 200 OK"




> Entering new AgentExecutor chain...


2024-02-03 19:21:40,071:INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



Invoking: `get_more_context` with `{'query': 'Napoleon und franz"}sische Revolution'}`




2024-02-03 19:21:40,515:INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-02-03 19:21:40,653:INFO - HTTP Request: POST https://smxwnqdpxcvclxhbvhsf.supabase.co/rest/v1/rpc/match_documents_with_uuid_filter?limit=20 "HTTP/1.1 200 OK"


2024-02-03 19:21:41,114:INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



Invoking: `tavily_search_results_json` with `{'query': 'Napoleon und die franz\\u00f6sische Revolution'}`
responded: Ich habe zusätzliche Informationen zu Napoleon und der französischen Revolution in meiner Datenbank gefunden. Ich werde die relevanten Informationen für dich abrufen und sie in die Unterrichtsvorbereitung einbeziehen.

[{'url': 'https://studyflix.de/geschichte/napoleon-bonaparte-5053', 'content': 'Französische Revolution Napoleon Mit\xa0der Französischen Revolution  Seinen Durchbruch erreichte Napoleon mit der Französischen Revolution  das Direktorium und erklärte die Französische Revolution für beendet. Dies war der Anfang der\xa0Napoleonischen Zeit.  Napoleon Bonaparte HerrschaftNapoleon Bonaparte (1769 - 1821) war ein berühmter Kaiser von Frankreich und erfolgreicher Kriegsherr.Sein Ziel war die Eroberung ganz Europas — was ihm durch sein militärisches Geschick auch beinahe gelang.. Seinen Durchbruch erreichte Napoleon mit der Französischen Revolution. Dabei stürzte 

2024-02-03 19:22:00,240:INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Hier sind einige relevante Informationen zu Napoleon und der französischen Revolution, die ich gefunden habe:

1. Napoleon erreichte seinen Durchbruch mit der französischen Revolution und erklärte die Revolution für beendet, was den Anfang der Napoleonischen Zeit markierte.

2. Napoleon Bonaparte (1769 - 1821) war ein berühmter Kaiser von Frankreich und ein erfolgreicher Kriegsherr. Sein Ziel war die Eroberung ganz Europas, was ihm fast gelang.

3. 1789 markierte den Beginn der Französischen Revolution, und 1814 wurde Napoleon abgesetzt.

4. Die Französische Revolution war eine Zeit großer gesellschaftlicher und politischer Umwälzungen in Frankreich, die den Zusammenbruch der Monarchie, die Gründung der Ersten Französischen Republik und den Aufstieg Napoleon Bonapartes umfasste.

Diese Informationen werden in die Unterrichtsvorbereitung aufgenommen, um die Schülerinnen und Schüler über Napoleon und die Bedeutung der französischen Revolution zu informieren.

> Finished chain.


In [5]:
instructions = "something"

In [6]:
base_prompt = hub.pull("hwchase17/openai-functions-agent")
prompt = base_prompt.partial(instructions=instructions)

In [7]:
prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'instructions': 'something'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')), MessagesPlaceholder(variable_name='chat_history', optional=True), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['in

In [8]:
prompt = hub.pull("c-7-s/openai-functions-agent-education")

In [9]:
school_type, subject, topic, grade, state, keywords, context = unpack_prompt_input(input_dict_1)

In [10]:
prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'chat_history', 'input', 'keywords', 'retrieved_context', 'topic'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input', 'keywords', 'retrieved_context', 'topic'], template='Du bist ein Assistent für Lehrkräfte und deine Aufgabe ist Unterricht strukturiert, detailliert und fachlich 

In [6]:
initialize_environment_variables("../.env")

# Initialize Supabase Client
supabase_client = initialize_subabase_client()

# Initialize OpenAI Client
_openai_client = initialize_openai_client()

# Initialize Tavily Client
_tavily_client = initialize_tavily_client()  

school_type, subject, topic, grade, state, keywords, context = unpack_prompt_input(input_dict_2)

task = create_task_string(school_type, subject, topic, grade, state)

embedding = OpenAIEmbeddings()
vector_store = initialize_vector_store(supabase_client, embedding, interface_config.upload_table_name, interface_config.supabase_match_function)

retrieved_context = vector_store.similarity_search(task, k=interface_config.context_k, filter=context)

prompt = hub.pull("c-7-s/openai-functions-agent-education")

search = TavilySearchAPIWrapper()
tavily_tool = TavilySearchResults(api_wrapper=search, max_results=interface_config.tavily_max_response)

wrapper = DuckDuckGoSearchAPIWrapper(region="de-de", time="d", max_results=5)
duckduck = DuckDuckGoSearchResults(api_wrapper=wrapper)

api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=100)
wikipedia = WikipediaQueryRun(api_wrapper=api_wrapper)

retriever = vector_store.as_retriever(search_type=interface_config.retriever_search_type, 
                                        search_kwargs={
                                            "k": interface_config.retriever_k,
                                            'lambda_mult': interface_config.retriever_mult, 
                                            'filter': {'uuid': context}
                                            }
                                        )

retriever_tool = create_retriever_tool(
retriever,
"get_more_context",
"A personal library optimized for user defined context information. Useful for when you need to know more about the given context. Input should be a search query.",
)

tools = [retriever_tool, tavily_tool, duckduck, wikipedia]

llm = ChatOpenAI(model_name=interface_config.gpt_model_version, temperature=interface_config.gpt_model_temperature)
agent = create_openai_functions_agent(llm, tools, prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

response = agent_executor.invoke({"input": task, "topic": topic, "retrieved_context": retrieved_context, "keywords": keywords, "chat_history": []})

2024-02-03 19:49:41,045:INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-02-03 19:49:41,289:INFO - HTTP Request: POST https://smxwnqdpxcvclxhbvhsf.supabase.co/rest/v1/rpc/match_documents_with_uuid_filter?limit=1 "HTTP/1.1 200 OK"




> Entering new AgentExecutor chain...


2024-02-03 19:49:42,237:INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


{
    "learn_goals": "Die Schüler sollen die wichtigsten Stationen von Napoleons Leben und Wirken kennenlernen und verstehen.",
    "table_data": [
        {
            "title": "Einführung und Vorbereitung",
            "duration": "5min",
            "content": "Kurze Einführung in das Thema Napoleon, Erklärung des historischen Kontexts und seiner Bedeutung für die französische Revolution."
        },
        {
            "title": "Napoleons Aufstieg zur Macht",
            "duration": "10min",
            "content": "Vorstellung und Erklärung von Napoleons Aufstieg zur Macht, Eroberungen und politischen Reformen."
        },
        {
            "title": "Napoleons Niederlage und Sturz",
            "duration": "10min",
            "content": "Behandlung von Napoleons Niederlage, Exil und Sturz, Diskussion über die Auswirkungen auf Europa und die Welt."
        },
        {
            "title": "Napoleons Vermächtnis",
            "duration": "10min",
            "content": "Anal